In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.grid_search import RandomizedSearchCV

/home/mike/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/mike/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
df = pd.read_csv("./work_dir/my_midterm_train.csv")

In [3]:
y=df['y']
X=df.drop(['y'], axis=1)

In [4]:
X.shape

(160000, 67)

In [5]:
y.shape

(160000,)

In [6]:
#setup our ML Pipe
scaler = RobustScaler()
logit = LogisticRegression(n_jobs=4)
pipe=Pipeline([('scaler', scaler), ('logit',logit)])

In [7]:
#setup grid search
hyperparameters = { 'logit__C':np.arange(.01, 10, .1)}
search = RandomizedSearchCV(pipe, hyperparameters, cv=5, scoring='roc_auc')

In [8]:
search.fit(X,y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=Pipeline(steps=[('scaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('logit', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=4,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'logit__C': array([ 0.01,  0.11, ...,  9.81,  9.91])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=0)

In [9]:
search.best_params_

{'logit__C': 6.4100000000000001}

In [10]:
search.best_score_

0.6336946031628996

In [11]:
## Make Prediction to get on the board

In [13]:
kaggle_test = pd.read_csv("./work_dir/my_midterm_kaggle_submission.csv")
#selected_features = pd.read_csv("./work_dir/feature_support.csv")

In [22]:
kaggle_test_selected = kaggle_test.ix[:, selected_features['0'].values]

In [15]:
prediction = pd.DataFrame(search.best_estimator_.predict_proba(kaggle_test)[:,1])

In [16]:
prediction.columns=['y']

In [17]:
prediction.to_csv("logit_model_prediction.csv", index_label="Id")